In [ ]:
# Load dataset
import pandas as pd
df = pd.read_csv("Corrected_PCB_Quality_Data.csv")

# Define target and features
target = "PCB_Status"
features = df.columns.drop(["PCB_Status", "Defect Type"])

# Encode target (OK = 1, Faulty = 0)
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

# Split features by type
categorical_cols = df[features].select_dtypes(include="object").columns.tolist()
numerical_cols = df[features].select_dtypes(include=["int64", "float64"]).columns.tolist()

# Preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])


In [ ]:
# Define models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Split data
from sklearn.model_selection import train_test_split
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Track best model
best_model = None
best_score = 0
best_name = ""

# Train and evaluate
from sklearn.metrics import accuracy_score
for name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")
    
    if acc > best_score:
        best_model = pipeline
        best_score = acc
        best_name = name

# Save best model
joblib.dump(best_model, "best_pcb_model.joblib")
print(f"\n✅ Best Model: {best_name} with Accuracy: {best_score:.4f}")


RandomForest Accuracy: 1.0000
DecisionTree Accuracy: 1.0000
LogisticRegression Accuracy: 1.0000
XGBoost Accuracy: 1.0000

✅ Best Model: RandomForest with Accuracy: 1.0000


c:\Users\sheet\Downloads\OneDrive\Desktop\PCB defect prediction\pcb_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:18:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
# Load saved model
import joblib
model = joblib.load("best_pcb_model.joblib")

# Sample input
sample_input = pd.DataFrame({
    'HCl (%)': [3.5],
    'H2SO4 (%)': [3.0],
    'CuSO4 (%)': [1.2],
    'pH Value': [5.0],
    'Temperature (°C)': [45.0],
    'Etching Time (s)': [110],
    'Capacitance (pF)': [30.0],
    'Insulation Resistance (MΩ)': [60.0],
    'Impedance Match (Ω)': [50.0],
    'Dielectric Breakdown (kV)': [3.5],
    'Board Thickness (mm)': [1.3],
    'Board Width (mm)': [150],
    'Board Length (mm)': [200],
    'Hole Diameter Deviation (mm)': [0.05],
    'Pad-to-Hole Alignment (mm)': [0.04],
    'Silkscreen Clarity': ['Clear'],            # ❌ Logic should mark as Faulty
    'Solderability Test': ['Pass'],
    'Dimension Accuracy': ['Accurate'],
    'Frequency Test Result': ['Pass'],
    'Defect Type': ['None']                     # Still says OK, but logic overrides
})

# Prepare input
model_input = sample_input.drop(columns=["Defect Type"])
model_prediction = model.predict(model_input)[0]  # 1 = OK, 0 = Faulty

# Apply override logic
def override_faulty(row):
    return (
        row['Silkscreen Clarity'] == 'Blurred' or
        row['Solderability Test'] == 'Fail' or
        row['Dimension Accuracy'] == 'Inaccurate' or
        row['Frequency Test Result'] == 'Fail' or
        row['Defect Type'] != 'None'
    )

if override_faulty(sample_input.iloc[0]):
    final_prediction = 0
else:
    final_prediction = model_prediction

# Show results
predicted_label = "OK" if final_prediction == 1 else "Faulty"
print("Model Prediction:", "OK" if model_prediction == 1 else "Faulty")
print("Final Corrected Prediction:", predicted_label)


Model Prediction: OK
Final Corrected Prediction: OK
